# Описание

В этом конкурсе kagle нужно определить конкретного пользователя сети Интернет по сессии посещения веб-сайтов. В каждой сессии может быть от 1 до 10 сайтов – количество сайтов ограничено длиной сессии.

Данные для конкурса собраны с прокси-серверов Университета Блеза Паскаля и взяты из статьи, которая описывает методы поиска Alice.

А мы будем решать задачу методами машинного обучения как задачу классификации.

**Постановка задачи**

Для каждой сессии нужно предсказать, принадлежит ли сессия Alice (метка «1»), или нет (метка «0»).

**Метрика оценки решений**

Целевая метрика – ROC AUC.

## 1. Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
from matplotlib import pyplot as plt

import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

## 2. Загрузка данных

In [2]:
train_df = pd.read_csv('train_sessions.csv',
                       index_col='session_id', parse_dates=['time1'])
test_df = pd.read_csv('test_sessions.csv',
                      index_col='session_id', parse_dates=['time1'])

train_df = train_df.sort_values(by='time1')

train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


## 3. Подготовка данных

### 3.1. Изменим тип столбцов site1, ..., site10 на integer и заполним NA-значения нулями

In [3]:
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

### 3.2. Загрузим словарь веб-сайтов

In [4]:
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

### 3.3 Создаём фрейм данных для словаря

In [5]:
sites_dict = pd.DataFrame(list(site_dict.keys()), index=list(site_dict.values()), columns=['site'])
print(u'Websites total:', sites_dict.shape[0])
sites_dict.head()

Websites total: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [6]:
print(test_df.shape, train_df.shape)

(82797, 20) (253561, 21)


In [7]:
y_train = train_df['target'].values

### 3.4. Объединяем фрейм исходных данных

In [8]:
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

### 3.5. Индекс для разделения наборов обучающих и тестовых данных

In [9]:
idx_split = train_df.shape[0]

### 3.6. Преобразуем данные в формат, который может быть передан в CountVectorizer.

In [10]:
train_df[sites].fillna(0).to_csv('train_sessions_text.txt', 
                                 sep=' ', index=None, header=None)
test_df[sites].fillna(0).to_csv('test_sessions_text.txt', 
                                sep=' ', index=None, header=None)

### 3.7. Установим с его помощью CountVectorizer и данные trasfrom.

In [11]:
%%time
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)
with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)
print(X_train.shape, X_test.shape)

(253561, 50000) (82797, 50000)
CPU times: total: 7.98 s
Wall time: 8 s


## 4. Обучение первой модели

In [12]:
def get_auc_lr_valid(X, y, C=1.0, seed=17, ratio = 0.9):
    # Разделим данные на обучающий и проверочный наборы
    idx = int(round(X.shape[0] * ratio))
    # Обучение классификатору
    lr = LogisticRegression(C=C, random_state=seed, solver='lbfgs', max_iter=500).fit(X[:idx, :], y[:idx])
    # Прогноз для набора проверки
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # Рассчет качества
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [13]:
X_train.shape, y_train.shape

((253561, 50000), (253561,))

### 4.1. Вычисляем метрику по набору проверки

In [14]:
%%time
print(get_auc_lr_valid(X_train, y_train))

0.9132504504635064
CPU times: total: 50.6 s
Wall time: 6.53 s


### 4.2. Функция для записи прогнозов в файл

In [15]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [16]:
# Обучим модель на всем наборе обучающих данных
# Параметр C=1 по умолчанию, но здесь мы устанавливаем его явно
lr = LogisticRegression(C=1.0, random_state=17, solver='lbfgs', max_iter=500).fit(X_train, y_train)

# Делаем прогноз для набора тестовых данных
y_test = lr.predict_proba(X_test)[:, 1]

# Запишим это в файл, который можно было бы отправить
write_to_submission_file(y_test, 'baseline_1.csv')